In [1]:
import bridgestan as bs
from pathlib import Path
import os
from utils import *
import json
import pickle
import arviz as az
import numpy as np
from tqdm import tqdm
pkey=1
n_repeat=100
evaluating_model='DirichletSymmetric'
data_file = lambda *, pkey, n_repeat, transform, evaluating_model: f"/mnt/home/mjhajaria/ceph/sampling_results/simplex/{transform}/{evaluating_model}/samples_{pkey}_{n_repeat}.nc"
stan_filename = lambda *, transform, evaluating_model: f"stan_models/{transform}_{evaluating_model}.stan"
output_filename = lambda *, transform, evaluating_model, x, pkey: f'output/{transform}/{evaluating_model}/{x}_{pkey}.npy'

# with open(f"target_densities/param_map_{evaluating_model}.pkl", "rb") as f:
#     param_map = pickle.load(f)

# for i in range(1,10):
#     with open(f'utils/dirichletsymmetricdata/data_{i}.json', 'w') as fp:
#         json.dump({'alpha':param_map[i], 'N': len(param_map[i])}, fp)


# pkeys=[1,2,3,4,5,6,7,8,9]

with open(stan_filename, 'w') as f:
    f.write(f'#include target_densities/DirichletSymmetric.stan{os.linesep}#include transforms/{transform}.stan{os.linesep}')
    f.close()

# x = data_file(pkey=pkey, n_repeat=100, 
#         transform=transform, evaluating_model='DirichletSymmetric')

/mnt/home/mjhajaria/miniconda3/envs/stan/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for transform in list_transforms():
    file=stan_filename(transform=transform, evaluating_model=evaluating_model)
    data= f'utils/dirichletsymmetricdata/data_{pkey}.json'
    bsmodel = bs.StanModel.from_stan_file(file, data, stanc_args=[f"--include-paths='/mnt/home/mjhajaria/simplex-transforms/'"])
    n=bsmodel.param_unc_num()
    m=bsmodel.param_num()
    print(bsmodel.param_names())
    print(pkey,m , n)

['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9', 'y.10']
1 10 10
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9
['y.1', 'y.2', 'y.3', 'y.4', 'y.5', 'y.6', 'y.7', 'y.8', 'y.9']
1 9 9


In [ ]:
idata = az.extract( az.from_netcdf(x).posterior)

In [ ]:
idata.data_vars.items()

In [ ]:
x = az.from_netcdf(x).posterior.x.values
data = x.reshape((x.shape[0]*x.shape[1], x.shape[2]))

In [ ]:
az.from_netcdf(x).posterior

In [ ]:
for idx, row in enumerate(data):
    theta = bsmodel.param_unconstrain(row)
    bsmodel.log_density_hessian(theta, out_grad=grad[idx], out_hess=hessian[idx])
np.save(output_filename(transform=transform, evaluating_model=evaluating_model, pkey=pkey, x="hessian"), hessian)
np.save(output_filename(transform=transform, evaluating_model=evaluating_model, 
                        pkey=pkey, x="grad"), grad)